In [3]:
import polars as pl
from pathlib import Path

In [2]:
# Ruta a la carpeta de los CSV
csv_folder = Path("../data/dataAtenciones")  # Ajusta el path si es necesario

# Lista de archivos CSV
csv_files = list(csv_folder.glob("*.csv"))
print(csv_files)

[WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2017_ATENCIONES.csv'), WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2018_ATENCIONES.csv'), WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2019_ATENCIONES.csv'), WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2020_ATENCIONES.csv'), WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2021_01_06_ATENCIONES.csv'), WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2021_07_12_ATENCIONES.csv'), WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2022_01_06_ATENCIONES.csv'), WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2022_07_12_ATENCIONES.csv'), WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2023_01_06_ATENCIONES.csv'), WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2023_07_12_ATENCIONES.csv'), WindowsPath('../data/dataAtenciones/OPENDATA_DS_01_2024_01_06_ATENCIONES.csv')]


In [3]:
import csv
from io import StringIO

In [4]:
# Lee todos los CSV y concatena en un solo DataFrame de Polars
df_list = []
for f in csv_files:
    # Lee el archivo y filtra filas mal formateadas usando el módulo csv
    with open(f, encoding="utf-8") as infile:
        valid_lines = []
        reader = csv.reader(infile)
        header = next(reader)
        valid_lines.append(",".join(header))
        for row in reader:
            try:
                # Si la fila tiene el mismo número de columnas que el header, la consideramos válida
                if len(row) == len(header):
                    valid_lines.append(",".join([str(x) for x in row]))
            except Exception:
                continue
        # Crea un StringIO con las filas válidas
        csv_buffer = StringIO("\n".join(valid_lines))
        df = pl.read_csv(
            csv_buffer,
            dtypes={
                "COD_IPRESS": pl.Utf8,
                "COD_SERVICIO": pl.Utf8
            },
            ignore_errors=True
        )
        if df.height > 0:
            df_list.append(df)

# Filtra DataFrames vacíos antes de concatenar
df = pl.concat(df_list)

C:\Users\Gianfranco\AppData\Local\Temp\ipykernel_19048\439311245.py:19: DeprecationWarning: the argument `dtypes` for `read_csv` is deprecated. It was renamed to `schema_overrides` in version 0.20.31.
  df = pl.read_csv(


In [5]:
df.write_parquet("../data/dataAtenciones_unificado.parquet")

In [2]:
# Lee el archivo parquet
df = pl.read_parquet("../data/dataAtenciones_unificado.parquet")

In [3]:
#Muestra las primeras 5 filas
df.head(5)

AÑO,MES,REGION,PROVINCIA,UBIGEO_DISTRITO,DISTRITO,COD_UNIDAD_EJECUTORA,DESC_UNIDAD_EJECUTORA,COD_IPRESS,IPRESS,NIVEL_EESS,PLAN_SEGURO,COD_SERVICIO,DESC_SERVICIO,SEXO,GRUPO_EDAD,ATENCIONES
i64,i64,str,str,i64,str,i64,str,str,str,str,str,str,str,str,str,i64
2017,2,"""AMAZONAS""","""RODRÍGUEZ DE MENDOZA""",10609,"""OMIA""",725,"""REGION AMAZONAS - SALUD""","""0000004946""","""OMIA""","""I""","""SIS GRATUITO""","""011""","""EXÁMENES DE LABORATORIO COMPLE…","""FEMENINO""","""30 - 59 AÑOS""",3
2017,2,"""AMAZONAS""","""RODRÍGUEZ DE MENDOZA""",10609,"""OMIA""",725,"""REGION AMAZONAS - SALUD""","""0000004946""","""OMIA""","""I""","""SIS GRATUITO""","""021""","""PREVENCION DE CARIES""","""FEMENINO""","""00 - 04 AÑOS""",1
2017,2,"""ÁNCASH""","""HUAYLAS""",21207,"""PUEBLO LIBRE""",744,"""REGION ANCASH - SALUD CARAZ""","""0000001611""","""CENTRO DE SALUD PUEBLO LIBRE""","""I""","""SIS GRATUITO""","""056""","""CONSULTA EXTERNA""","""FEMENINO""","""00 - 04 AÑOS""",37
2017,2,"""ÁNCASH""","""HUAYLAS""",21207,"""PUEBLO LIBRE""",744,"""REGION ANCASH - SALUD CARAZ""","""0000001611""","""CENTRO DE SALUD PUEBLO LIBRE""","""I""","""SIS GRATUITO""","""075""","""ATENCIÓN EXTRAMURAL RURAL (VIS…","""FEMENINO""","""00 - 04 AÑOS""",13
2017,2,"""ÁNCASH""","""HUAYLAS""",21207,"""PUEBLO LIBRE""",744,"""REGION ANCASH - SALUD CARAZ""","""0000001611""","""CENTRO DE SALUD PUEBLO LIBRE""","""I""","""SIS INDEPENDIENTE""","""071""","""APOYO AL DIAGNÓSTICO""","""FEMENINO""","""18 - 29 AÑOS""",2


In [4]:
#muestra el tipo de dato de cada columna
print(df.dtypes)

[Int64, Int64, String, String, Int64, String, Int64, String, String, String, String, String, String, String, String, String, Int64]


In [5]:
for name, dtype in zip(df.columns, df.dtypes):
    print(f"{name}: {dtype}")

AÑO: Int64
MES: Int64
REGION: String
PROVINCIA: String
UBIGEO_DISTRITO: Int64
DISTRITO: String
COD_UNIDAD_EJECUTORA: Int64
DESC_UNIDAD_EJECUTORA: String
COD_IPRESS: String
IPRESS: String
NIVEL_EESS: String
PLAN_SEGURO: String
COD_SERVICIO: String
DESC_SERVICIO: String
SEXO: String
GRUPO_EDAD: String
ATENCIONES: Int64


In [6]:
# Selecciona las columnas de interés y renombra 'AÑO' a 'ANIO'
df_filtrado = df.select([
    pl.col("AÑO").alias("ANIO"),
    "REGION",
    "PROVINCIA",
    "UBIGEO_DISTRITO",
    "DISTRITO",
    "COD_UNIDAD_EJECUTORA",
    "DESC_UNIDAD_EJECUTORA",
    "COD_IPRESS",
    "IPRESS",
    "COD_SERVICIO",
    "DESC_SERVICIO",
    "ATENCIONES"
])

In [7]:
# Mostrar las primeras 5 filas
df_filtrado.head(5)

ANIO,REGION,PROVINCIA,UBIGEO_DISTRITO,DISTRITO,COD_UNIDAD_EJECUTORA,DESC_UNIDAD_EJECUTORA,COD_IPRESS,IPRESS,COD_SERVICIO,DESC_SERVICIO,ATENCIONES
i64,str,str,i64,str,i64,str,str,str,str,str,i64
2017,"""AMAZONAS""","""RODRÍGUEZ DE MENDOZA""",10609,"""OMIA""",725,"""REGION AMAZONAS - SALUD""","""0000004946""","""OMIA""","""011""","""EXÁMENES DE LABORATORIO COMPLE…",3
2017,"""AMAZONAS""","""RODRÍGUEZ DE MENDOZA""",10609,"""OMIA""",725,"""REGION AMAZONAS - SALUD""","""0000004946""","""OMIA""","""021""","""PREVENCION DE CARIES""",1
2017,"""ÁNCASH""","""HUAYLAS""",21207,"""PUEBLO LIBRE""",744,"""REGION ANCASH - SALUD CARAZ""","""0000001611""","""CENTRO DE SALUD PUEBLO LIBRE""","""056""","""CONSULTA EXTERNA""",37
2017,"""ÁNCASH""","""HUAYLAS""",21207,"""PUEBLO LIBRE""",744,"""REGION ANCASH - SALUD CARAZ""","""0000001611""","""CENTRO DE SALUD PUEBLO LIBRE""","""075""","""ATENCIÓN EXTRAMURAL RURAL (VIS…",13
2017,"""ÁNCASH""","""HUAYLAS""",21207,"""PUEBLO LIBRE""",744,"""REGION ANCASH - SALUD CARAZ""","""0000001611""","""CENTRO DE SALUD PUEBLO LIBRE""","""071""","""APOYO AL DIAGNÓSTICO""",2


In [8]:
# Mostrar los tipos de dato
df_filtrado.dtypes

[Int64,
 String,
 String,
 Int64,
 String,
 Int64,
 String,
 String,
 String,
 String,
 String,
 Int64]

In [9]:
# UBIGEO_DISTRITO <-> DISTRITO
df_ubigeo_distrito = df_filtrado.select(["UBIGEO_DISTRITO", "DISTRITO"]).unique()

# COD_UNIDAD_EJECUTORA <-> DESC_UNIDAD_EJECUTORA
df_cod_ue_desc = df_filtrado.select(["COD_UNIDAD_EJECUTORA", "DESC_UNIDAD_EJECUTORA"]).unique()

# COD_IPRESS <-> IPRESS
df_cod_ipress_ipress = df_filtrado.select(["COD_IPRESS", "IPRESS"]).unique()

# COD_SERVICIO <-> DESC_SERVICIO
df_cod_servicio_desc = df_filtrado.select(["COD_SERVICIO", "DESC_SERVICIO"]).unique()

In [11]:
print(df_ubigeo_distrito.head())

shape: (5, 2)
┌─────────────────┬───────────┐
│ UBIGEO_DISTRITO ┆ DISTRITO  │
│ ---             ┆ ---       │
│ i64             ┆ str       │
╞═════════════════╪═══════════╡
│ 80207           ┆ SANGARARA │
│ 150506          ┆ COAYLLO   │
│ 151030          ┆ TOMAS     │
│ 120127          ┆ QUICHUAY  │
│ 130705          ┆ SAN JOSÉ  │
└─────────────────┴───────────┘


In [12]:
print(df_cod_ue_desc.head())

shape: (5, 2)
┌──────────────────────┬─────────────────────────────────┐
│ COD_UNIDAD_EJECUTORA ┆ DESC_UNIDAD_EJECUTORA           │
│ ---                  ┆ ---                             │
│ i64                  ┆ str                             │
╞══════════════════════╪═════════════════════════════════╡
│ 1059                 ┆ REGION SAN MARTIN - SALUD HUAL… │
│ 765                  ┆ REGION AREQUIPA - SALUD         │
│ 830                  ┆ REGION JUNIN - SALUD JUNIN      │
│ 787                  ┆ REGION CAJAMARCA - SALUD CUTER… │
│ 1497                 ┆ GOB. REG. APURIMAC - RED DE SA… │
└──────────────────────┴─────────────────────────────────┘


In [13]:
print(df_cod_ipress_ipress.head())

shape: (5, 2)
┌────────────┬────────────────────────────────┐
│ COD_IPRESS ┆ IPRESS                         │
│ ---        ┆ ---                            │
│ str        ┆ str                            │
╞════════════╪════════════════════════════════╡
│ 0000028772 ┆ PUESTO DE SALUD NUEVO PROGRESO │
│ 0000005732 ┆ CERRO CULEBRA                  │
│ 0000001831 ┆ PUESTO DE SALUD  HUACHIS       │
│ 0000006109 ┆ JESUS PODEROSO                 │
│ 0000000631 ┆ COCHARCAS                      │
└────────────┴────────────────────────────────┘


In [ ]:
df_cod_ipress_ipress.write_csv("../data/cod_ipress_ipress.csv")

In [14]:
print(df_cod_servicio_desc.head())

shape: (5, 2)
┌──────────────┬─────────────────────────────────┐
│ COD_SERVICIO ┆ DESC_SERVICIO                   │
│ ---          ┆ ---                             │
│ str          ┆ str                             │
╞══════════════╪═════════════════════════════════╡
│ 062          ┆ ATENCIÓN POR EMERGENCIA         │
│ 065          ┆ INTERNAMIENTO EN EESS SIN INTE… │
│ 011          ┆ EXÁMENES DE LABORATORIO COMPLE… │
│ 111          ┆ ASIGNACIÓN POR ALIMENTACIÓN     │
│ 300          ┆ TELEMEDICINA                    │
└──────────────┴─────────────────────────────────┘


In [16]:
df_cod_servicio_desc.write_csv("../data/df_cod_servicio_desc.csv")

In [ ]:
import pandas as pd

# Diccionario completo de mapeo
mapeo_especialidades = {
    "062": "Medicina de Emergencia",
    "065": "Medicina Interna",
    "011": "Ginecología/Obstetricia",
    "111": "Nutrición",
    "300": "Medicina General",
    "069": "Hematología",
    "007": "Nutrición",
    "024": "Ginecología",
    "904": "Medicina Familiar",
    "911": "Odontología",
    "029": "Neonatología/Pediatría",
    "026": "Ginecología/Infectología",
    "010": "Ginecología/Obstetricia",
    "066": "Cirugía General",
    "909": "Medicina General",
    "907": "Medicina General",
    "018": "Ginecología",
    "022": "Psiquiatría/Salud Mental",
    "020": "Odontología",
    "008": "Medicina Preventiva",
    "900": "Odontología",
    "910": "Medicina General",
    "068": "Medicina Crítica/Cuidados Intensivos",
    "005": "Nutrición",
    "016": "Pediatría",
    "019": "Oftalmología",
    "902": "Ginecología",
    "013": "Ginecología/Obstetricia",
    "023": "Urología",
    "063": "Medicina de Emergencia",
    "070": "Odontología",
    "015": "Ginecología/Obstetricia",
    "118": "Pediatría",
    "S01": "Medicina General",
    "009": "Ginecología/Obstetricia",
    "903": "Geriatría",
    "906": "Medicina General",
    "053": "Infectología/Pediatría",
    "S02": "Pediatría",
    "051": "Neonatología",
    "017": "Medicina del Adolescente",
    "901": "Medicina General",
    "027": "Infectología/Pediatría",
    "001": "Pediatría",
    "067": "Cirugía General",
    "059": "Odontología",
    "025": "Ginecología/Mastología",
    "057": "Odontología",
    "050": "Neonatología",
    "021": "Odontología",
    "074": "Infectología/Dermatología",
    "119": "Pediatría",
    "052": "Neonatología/Cirugía Pediátrica",
    "054": "Ginecología/Obstetricia",
    "071": "Medicina General",
    "056": "Medicina General",
    "200": "Fisiatría/Rehabilitación",
    "060": "Medicina Familiar",
    "064": "Cirugía General",
    "002": "Neonatología",
    "058": "Odontología",
    "117": "Medicina de Emergencia",
    "061": "Medicina General",
    "075": "Medicina Familiar",
    "908": "Medicina Familiar",
    "055": "Ginecología/Obstetricia"
}

# Cargar el archivo CSV (asegúrate de que la ruta sea correcta)
df = pd.read_csv("../data/df_cod_servicio_desc.csv")  # Reemplaza con tu ruta

# Agregar la columna ESPECIALIDAD_MEDICA
df["ESPECIALIDAD_MEDICA"] = df["COD_SERVICIO"].map(mapeo_especialidades)

# Guardar el nuevo CSV (opcional)
df.to_csv("../data/servicios_con_especialidades.csv", index=False)

# Mostrar el DataFrame
df

,COD_SERVICIO,DESC_SERVICIO,ESPECIALIDAD_MEDICA
0,062,ATENCIÓN POR EMERGENCIA,Medicina de Emergencia
1,065,INTERNAMIENTO EN EESS SIN INTERVENCIÓN QUIRÚRGICA,Medicina Interna
2,011,EXÁMENES DE LABORATORIO COMPLETO DE LA GESTANTE,Ginecología/Obstetricia
3,111,ASIGNACIÓN POR ALIMENTACIÓN,Nutrición
4,300,TELEMEDICINA,Medicina General
...,...,...,...
61,117,TRASLADO DE EMERGENCIA,Medicina de Emergencia
62,061,ATENCIÓN EN TÓPICO,Medicina General
63,075,ATENCIÓN EXTRAMURAL RURAL (VISITA DOMICILIARIA),Medicina Familiar
64,908,ATENCIÓN DOMICILIARIA,Medicina Familiar


In [20]:
import polars as pl

# Lee el DataFrame principal
df = pl.read_parquet("../data/dataAtenciones_unificado.parquet")
df_filtrado = df.select([
    pl.col("AÑO").alias("ANIO"),
    "REGION",
    "PROVINCIA",
    "UBIGEO_DISTRITO",
    "DISTRITO",
    "COD_UNIDAD_EJECUTORA",
    "DESC_UNIDAD_EJECUTORA",
    "COD_IPRESS",
    "IPRESS",
    "COD_SERVICIO",
    "DESC_SERVICIO",
    "ATENCIONES"
])

# Lee el CSV de mapeo de especialidades
df_mapeo = pl.read_csv("../data/servicios_con_especialidades.csv")

# Une la especialidad médica al DataFrame principal
df_final = df_filtrado.join(df_mapeo, on=["COD_SERVICIO", "DESC_SERVICIO"], how="left")

# Muestra las primeras filas para verificar
print(df_final.head())

shape: (5, 13)
┌──────┬──────────┬────────────┬────────────┬───┬────────────┬────────────┬────────────┬───────────┐
│ ANIO ┆ REGION   ┆ PROVINCIA  ┆ UBIGEO_DIS ┆ … ┆ COD_SERVIC ┆ DESC_SERVI ┆ ATENCIONES ┆ ESPECIALI │
│ ---  ┆ ---      ┆ ---        ┆ TRITO      ┆   ┆ IO         ┆ CIO        ┆ ---        ┆ DAD_MEDIC │
│ i64  ┆ str      ┆ str        ┆ ---        ┆   ┆ ---        ┆ ---        ┆ i64        ┆ A         │
│      ┆          ┆            ┆ i64        ┆   ┆ str        ┆ str        ┆            ┆ ---       │
│      ┆          ┆            ┆            ┆   ┆            ┆            ┆            ┆ str       │
╞══════╪══════════╪════════════╪════════════╪═══╪════════════╪════════════╪════════════╪═══════════╡
│ 2017 ┆ AMAZONAS ┆ RODRÍGUEZ  ┆ 10609      ┆ … ┆ 011        ┆ EXÁMENES   ┆ 3          ┆ Ginecolog │
│      ┆          ┆ DE MENDOZA ┆            ┆   ┆            ┆ DE LABORAT ┆            ┆ ía/Obstet │
│      ┆          ┆            ┆            ┆   ┆            ┆ ORIO       ┆ 

In [21]:
import polars as pl

In [22]:
# 1. Lee el DataFrame principal
df = pl.read_parquet("../data/dataAtenciones_unificado.parquet")

In [23]:
# 2. Selecciona y renombra las columnas de interés
df_filtrado = df.select([
    pl.col("AÑO").alias("ANIO"),
    "MES",
    "COD_IPRESS",
    "IPRESS",
    "UBIGEO_DISTRITO",
    "DISTRITO",
    "REGION",
    "PROVINCIA",
    "ATENCIONES",
    "COD_SERVICIO"
])

In [24]:
# 3. Lee el CSV de mapeo de especialidades
df_mapeo = pl.read_csv("../data/servicios_con_especialidades.csv")  # Debe tener COD_SERVICIO y ESPECIALIDAD_MEDICA

In [25]:
# 4. Une la especialidad médica al DataFrame principal usando COD_SERVICIO
df_final = df_filtrado.join(df_mapeo.select(["COD_SERVICIO", "ESPECIALIDAD_MEDICA"]), on="COD_SERVICIO", how="left")

In [26]:
# 5. Guarda el resultado en CSV y Parquet
df_final.write_csv("../data/dataAtenciones_con_especialidad.csv")
df_final.write_parquet("../data/dataAtenciones_con_especialidad.parquet")

In [28]:
# (Opcional) Muestra las primeras filas para verificar
df_final.head()

ANIO,MES,COD_IPRESS,IPRESS,UBIGEO_DISTRITO,DISTRITO,REGION,PROVINCIA,ATENCIONES,COD_SERVICIO,ESPECIALIDAD_MEDICA
i64,i64,str,str,i64,str,str,str,i64,str,str
2017,2,"""0000004946""","""OMIA""",10609,"""OMIA""","""AMAZONAS""","""RODRÍGUEZ DE MENDOZA""",3,"""011""","""Ginecología/Obstetricia"""
2017,2,"""0000004946""","""OMIA""",10609,"""OMIA""","""AMAZONAS""","""RODRÍGUEZ DE MENDOZA""",1,"""021""","""Odontología"""
2017,2,"""0000001611""","""CENTRO DE SALUD PUEBLO LIBRE""",21207,"""PUEBLO LIBRE""","""ÁNCASH""","""HUAYLAS""",37,"""056""","""Medicina General"""
2017,2,"""0000001611""","""CENTRO DE SALUD PUEBLO LIBRE""",21207,"""PUEBLO LIBRE""","""ÁNCASH""","""HUAYLAS""",13,"""075""","""Medicina Familiar"""
2017,2,"""0000001611""","""CENTRO DE SALUD PUEBLO LIBRE""",21207,"""PUEBLO LIBRE""","""ÁNCASH""","""HUAYLAS""",2,"""071""","""Medicina General"""


In [4]:
# Lee el archivo ya preparado con especialidades
df = pl.read_parquet("../data/dataAtenciones_con_especialidad.parquet")

In [5]:
# Agrupa por ANIO, MES, COD_IPRESS, IPRESS y ESPECIALIDAD_MEDICA, sumando las atenciones
df_modelo = (
    df
    .group_by(["ANIO", "MES", "COD_IPRESS", "IPRESS", "ESPECIALIDAD_MEDICA"])
    .agg([
        pl.col("ATENCIONES").cast(pl.Int64).sum().alias("TOTAL_ATENCIONES")
    ])
)

In [9]:
# Calcula la cantidad de profesionales requeridos (mínimo 1 por especialidad)
df_modelo = df_modelo.with_columns(
    (pl.col("TOTAL_ATENCIONES") / 15).ceil().clip(1, None).alias("PROFESIONALES_REQUERIDOS")
)

In [10]:
# Guarda el resultado en CSV y Parquet
df_modelo.write_csv("../data/data_modelo_profesionales.csv")
df_modelo.write_parquet("../data/data_modelo_profesionales.parquet")

In [12]:
# (Opcional) Muestra las primeras filas para verificar
df_modelo.head()

ANIO,MES,COD_IPRESS,IPRESS,ESPECIALIDAD_MEDICA,TOTAL_ATENCIONES,PROFESIONALES_REQUERIDOS
i64,i64,str,str,str,i64,f64
2019,2,"""0000002604""","""PAMPAHUITE""","""Medicina Familiar""",31,3.0
2024,5,"""0000001843""","""PUESTO DE SALUD PAUCAS""","""Medicina del Adolescente""",20,2.0
2018,1,"""0000000501""","""ALTO PITOCUNA""","""Medicina General""",112,8.0
2024,1,"""0000000741""","""CEDRUYO""","""Nutrición""",1,1.0
2023,2,"""0000012148""","""SORTILEGIO""","""Medicina Preventiva""",5,1.0
